In [1]:
# Suggested data science package imports
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

# TrendMiner package import
import trendminer
from trendminer.trendminer_client import TrendMinerClient

import os

token = os.environ["KERNEL_USER_TOKEN"]

# Create TrendMiner API object
client = TrendMinerClient(token)

import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests

In [3]:
#Functions used throughout script.
#Do not modify...

def get_interpolated_data(tagname, start, end, steps):
    
    headers = {'Authorization': f'Bearer {os.environ["KERNEL_USER_TOKEN"]}'}
    
    params = {'tagName': tagname}
    
    r = requests.get(f'{os.environ["KERNEL_SERVER_URL"]}/hps/api/tags/details', params=params, headers=headers, verify=False)
    
    interpolation_type = r.json()['interpolationType']
    tagID = r.json()['id']
    
    request_body = {
        "tag": {
        "id": tagID,
        "shift": 0,
        "interpolationType": interpolation_type
      },
      "timePeriod": {
        "startDate": start,
        "endDate": end
      },
      "step": steps
    }

    r = requests.post(f'{os.environ["KERNEL_SERVER_URL"]}/compute/interpolatedData/paged', json=request_body, headers=headers, verify=False)
    
    data = r.json()
    
    result = data['values']
    
    # Check if there is a next start date
    while 'nextStartDate' in data:
        # Update the start date for the next request
        next_start = data['nextStartDate']
        
        request_body = {
            "tag": {
            "id": tagID,
            "shift": 0,
            "interpolationType": interpolation_type
          },
          "timePeriod": {
            "startDate": next_start,
            "endDate": end
          },
          "step": steps
        }
        
        r = requests.post(f'{os.environ["KERNEL_SERVER_URL"]}/compute/interpolatedData/paged', json=request_body, headers=headers, verify=False)
        
        data = r.json()
        
        result += data['values']
    
    return pd.DataFrame(result)


def get_tag_details(tagname):

    headers = {'Authorization': f'Bearer {os.environ["KERNEL_USER_TOKEN"]}'}
    
    params = {'tagName': tagname}
    
    r = requests.get(f'{os.environ["KERNEL_SERVER_URL"]}/hps/api/tags/details', params=params, headers=headers)
    
    return r.json()


def get_last_value(tagID, tagInterpolation):

    headers = {'Authorization': f'Bearer {os.environ["KERNEL_USER_TOKEN"]}'}
    
    params = {'timeSeriesId': tagID, 'interpolationType': tagInterpolation}
    
    r = requests.get(f'{os.environ["KERNEL_SERVER_URL"]}/compute/data/last-value', params=params, headers=headers)

    return r.json()

In [8]:
# Enter the information below:

#Product tag to be used later to filter timeseries data. E.g. TM-BP2-PRODUCT.1 can be equal to ALPHA, BETA, GAMMA, etc.
product_tag = "TM-BP2-PRODUCT.1" 

#Tags to pull timeseries data for. Include the same product tag defined earlier.
tags = ["TM-BP2-LEVEL.1", "TM-BP2-PRESSURE.1", product_tag]

#start and end timestamps to pull timeseries data. format yyyy-MM-ddTHH:mm:ss.000Z
start = "2019-01-15T07:00:02.000Z"
end = "2019-03-01T07:10:02.000Z"

#Interval resolution in seconds.
steps = 60 # Enter interval (s) per point


# Do not modify script below.
tag_details = get_tag_details(product_tag)
tag_current_value = get_last_value(tag_details['id'], tag_details['interpolationType'])
current_product = tag_current_value['value']

for tag in tags:    
    data = get_interpolated_data(tag, start, end, steps)
    data = data.set_index('ts')
    data['value'] = data['value'].astype(float).round(decimals=5)
    data = data.rename(columns={'value':tag})
    
    if tags.index(tag) == 0:
        tag_data_df = data
    else:
        tag_data_df =  tag_data_df.merge(data, how='outer', left_index=True, right_index=True)

tag_data_df_filter = tag_data_df[tag_data_df[product_tag]==current_product]

In [9]:
tag_data_df_filter.describe()

In [10]:
tag_data_df_filter.index = pd.to_datetime(tag_data_df_filter.index)

# Group by month
df_monthly = tag_data_df_filter.groupby(tag_data_df_filter.index.to_period("M")).mean()


In [11]:
df_monthly